In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Model

In [139]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [140]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels  # y_true
        self.prediction = self.predict()  # y_pred
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction        

In [141]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1     # "impurity" - "нечистота", степень неопределенности
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [142]:
# Расчет качества

def quality(left_labels, right_labels, current_gini):

    # доля выборки, ушедшей в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [143]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [144]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels, indexes=[]):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 5

    current_gini = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        if len(indexes)>0 and index not in indexes:
            continue
        t_values = [row[index] for row in data]
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [145]:
# Построение дерева с помощью рекурсивной функции
indexes = []
def build_tree(data, labels, max_deep=5, cur_deep=0, max_leafs=5, max_indexes=5, is_initial=True):
    global indexes
    if is_initial:
        indexes = []

    if max_deep > 0 and cur_deep >= max_deep:
        return Leaf(data, labels)
    
    if len(indexes) == max_indexes:
        quality, t, index = find_best_split(data, labels, indexes=indexes)
    else:
        quality, t, index = find_best_split(data, labels)
        if index not in indexes:
            indexes.append(index)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, max_deep=max_deep, cur_deep=cur_deep+1, max_indexes=max_indexes, is_initial=False)
    false_branch = build_tree(false_data, false_labels, max_deep=max_deep, cur_deep=cur_deep+1, max_indexes=max_indexes, is_initial=False)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [146]:
# Проход объекта по дереву для его классификации

def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [147]:
# Предсказание деревом для всего датасета

def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

# Data

In [4]:
df = pd.read_csv('train.csv')

In [5]:
TARGET_NAME = 'choose'
FEATURES = ['age', 'years_of_experience', 'lesson_price', 'mean_exam_points', 'qualification', 'physics', 'chemistry', 'biology', 'english', 'geography', 'history']
DROP_COLUMNS = ['Id']

# Data (balance)

In [96]:
df_choose_1 = df.loc[df['choose'] == 1][0: 1000]
df_choose_0 = df.loc[df['choose'] == 0][0: 1000]
df_balanced = df_choose_1.append(df_choose_0, ignore_index=True)
df_balanced = df_balanced.sample(frac=1)
X = df_balanced.drop(columns=TARGET_NAME)
X = X.drop(columns=DROP_COLUMNS)
y = df_balanced[TARGET_NAME]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Boosting

In [162]:
def get_error(pred, y):
    return sum(pred != y) / len(y)

In [163]:
def adaboost(X, y, N):

    # Размер выборки
    n_objects = len(X)

    # Запишем количество классов в переменную
    n_classes = len(np.unique((y)))

    # Начальные веса деревьев
    w = np.ones(n_objects) / n_objects

    # Деревья с весами будем записывать в список
    models = []

    for n in range(N):
        # Зададим дерево и обучим его
        my_tree = build_tree(X, y)

        predictions = predict(X, my_tree)
        e = get_error(predictions, y)
        # отбросим дерево, если его ошибка больше 0.5
        # Запишем условие в общем виде (применимо к небинарным классификаторам)
        if e >= 1 - 1/n_classes: 
            break

        # Вычислим вес для дерева
        alpha = 0.5 * np.log((1 - e) / e)

        # Найдем индексы правильно классифицированных элементов
        match = predictions == y

        # Увеличим веса для неправильно классифицированных элементов
        w[~match] *= np.exp(alpha)

        # Нормализуем веса
        w /= w.sum()

        # Добавим дерево с весом в список
        models.append((alpha, my_tree))
    
    return models

In [ ]:
N = 50

models = adaboost(X_train, y_train, N)

In [180]:
def predict_ada(X, models):
    
    n_classes = 2
    n_objects = len(X)
    
    # вначале обозначим предсказание нулевым массивом
    y_pred = np.zeros((n_objects, n_classes))
    
    for alpha, clf in models:
        prediction = predict(X, clf)
        # Для каждого предсказания будем прибавлять alpha к
        # элементу с индексом предсказанного класса
        y_pred[range(n_objects), prediction] += alpha
    
    # выберем индексы с максимальными суммарными весами -
    # получим предсказанные алгоритмом классы
    y_pred = np.argmax(y_pred, axis=1)
    
    return y_pred

print(f'Точность алгоритма на обучающей выборке: {(1 - get_error(predict_ada(X_train, models), y_train)) * 100:.3f}')

Точность алгоритма на обучающей выборке: 73.500


In [181]:
print(f'Точность алгоритма на тестовой выборке: {(1 - get_error(predict_ada(X_test, models), y_test)) * 100:.3f}')

Точность алгоритма на тестовой выборке: 71.500


# Save result to file

In [185]:
df_res = pd.read_csv('test.csv')
X_res = np.array(df_res.drop(columns=DROP_COLUMNS))
y_res = predict_ada(X_res, models)
with open('Vvashchenkov_sub.csv', 'w', encoding='utf-8') as f:
    f.write('Id,choose\n')
    idx = 10000
    for i in range(len(y_res)):
        f.write(f'{idx},{y_res[i]}\n')
        idx += 1